In [25]:
from scipy import stats
rvs1 = stats.norm.rvs(loc = 5,scale = 10,size = 500)
rvs2 = stats.norm.rvs(loc = 10,scale = 10,size = 500)
print(stats.ttest_ind(rvs1,rvs2))
import json
import numpy as np



Ttest_indResult(statistic=-8.033281728779778, pvalue=2.6620288643043905e-15)


In [40]:
# Load the eval_score
def extract_score_array(score_dict, n=None):
    max_n = score_dict['eval_score']['n_samples']
    if n is None or n > max_n:
        n = int(max_n)
    mse = []
    lpips = []
    dssim = []
    for i, img_name in enumerate(score_dict['each_image'].keys()):
        mse.append(float(score_dict['each_image'][img_name]['mse']))
        lpips.append(float(score_dict['each_image'][img_name]['lpips']))
        dssim.append(float(score_dict['each_image'][img_name]['dssim']))
        
        if i == n-1: break
    assert len(mse) == n
    assert len(lpips) == n
    assert len(dssim) == n
    
    return mse, lpips, dssim
       
        
with open('/data/mint/paper_evaluations/MP/pred/Ours/ours_128_Masked_Face_woclip+BgNoHead+shadow+arcface/ema_050000/out/eval_score_validset_n200.json', 'r') as fp:
    only_arcface = json.load(fp=fp)
mse_only_arcface, lpips_only_arcface, dssim_only_arcface = extract_score_array(only_arcface)   

with open('/data/mint/paper_evaluations/MP/pred/Ours/ours_128_Masked_Face_woclip+BgNoHead+shadow/ema_050000/out/eval_score_validset_n200.json', 'r') as fp:
    deca_arcface = json.load(fp=fp)
mse_deca_arcface, lpips_deca_arcface, dssim_deca_arcface = extract_score_array(deca_arcface)


print("[#] MSE : ", stats.ttest_ind(mse_1, mse_2))
print("[#] LPIPS : ", stats.ttest_ind(lpips_1, lpips_2))
print("[#] DSSIM : ", stats.ttest_ind(dssim_1, dssim_2))


[#] Stat-test : 
[#] Stat-test : 
[#] MSE :  Ttest_indResult(statistic=-2.488701037276761, pvalue=0.013228977921074202)
[#] LPIPS :  Ttest_indResult(statistic=1.593949104311001, pvalue=0.11174122571466671)
[#] DSSIM :  Ttest_indResult(statistic=0.5148355728333595, pvalue=0.6069537154249971)


In [73]:
def json_score(fn):
    with open(fn, 'r') as fp:
        out = json.load(fp=fp)
    mse, lpips, dssim = extract_score_array(out)
    return out, mse, lpips, dssim

only_arcface = '/data/mint/paper_evaluations/MP/score/ablation/non_spatial/eval_score_ours_128_Masked_Face_woclip+BgNoHead+shadow+arcface_validset_n200.json'
only_deca = '/data/mint/paper_evaluations/MP/score/ablation/non_spatial/eval_score_ours_128_Masked_Face_woclip+BgNoHead+shadow+deca_validset_n200.json'
deca_arcface = '/data/mint/paper_evaluations/MP/score/ablation/non_spatial/eval_score_ours_128_Masked_Face_woclip+BgNoHead+shadow_validset_n200.json'
only_shadow = '/data/mint/paper_evaluations/MP/score/ablation/non_spatial/eval_score_ours_128_Masked_Face_woclip+BgNoHead+only_shadow_validset_n200.json'

score_d = {'only_shadow':{'path':only_shadow},
           'only_deca':{'path':only_deca},
           'deca_arcface':{'path':deca_arcface},
           'only_arcface':{'path':only_arcface},
        }

for s in score_d.keys():
    print("[#] Loading : ", s)
    out, mse, lpips, dssim = json_score(score_d[s]['path'])
    score_d[s]['mse'] = mse
    score_d[s]['lpips'] = lpips
    score_d[s]['dssim'] = dssim
    score_d[s]['json'] = out
        

[#] Loading :  only_shadow
[#] Loading :  only_deca
[#] Loading :  deca_arcface
[#] Loading :  only_arcface


In [74]:
# Gen LATEX
for s in score_d.keys():
    mse_mu, mse_std = score_d[s]['json']['eval_score']['mse'].split('+-')
    mse_mu, mse_std = float(mse_mu), float(mse_std)
    
    lpips_mu, lpips_std = score_d[s]['json']['eval_score']['lpips'].split('+-')
    lpips_mu, lpips_std = float(lpips_mu), float(lpips_std)
    
    dssim_mu, dssim_std = score_d[s]['json']['eval_score']['dssim'].split('+-')
    dssim_mu, dssim_std = float(dssim_mu), float(dssim_std)
    
    fmt = f'& {dssim_mu:.4f}$\pm${dssim_std:.4f} & {mse_mu:.4f}$\pm${mse_std:.4f} & {lpips_mu:.4f}$\pm${lpips_std:.4f}'
    print(f'{s} ==> {fmt}')
    

only_shadow ==> & 0.1065$\pm$0.0391 & 0.0082$\pm$0.0091 & 0.0909$\pm$0.0454
only_deca ==> & 0.1023$\pm$0.0355 & 0.0074$\pm$0.0068 & 0.0847$\pm$0.0391
deca_arcface ==> & 0.1000$\pm$0.0344 & 0.0077$\pm$0.0073 & 0.0789$\pm$0.0329
only_arcface ==> & 0.1022$\pm$0.0329 & 0.0063$\pm$0.0052 & 0.0846$\pm$0.0350


In [ ]:

print(stats.ttest_ind(rvs1,rvs2))